In [14]:
import numpy as np
from numpy import *
import qutip
from qutip import *
from numpy.random import rand, randint
from qutip.measurement import measure, measurement_statistics
# from qutip.states import ket2dm, basis

In [15]:
class Node():
    def get_alpha(self):
        return self.alpha
    def get_children(self):
        return self.children

class DecisionTree():

    # this is an M-ary tree of depth N, i.e. it has M**(N+1) - 1 nodes in total
    # each node corresponds to a measurement history,
    # i.e. a list x_j = [k_0, k_1, ... k_j] of measurement outcomes.

    
    # initialize as as random. each node has value between 0 and 1/2 + i
    def __init__(self, M, N):
        self.alpha = 0.5*rand() + 1j*rand()
        if N == 1:
            self.children = None
        else:
            self.children = [DecisionTree(M, N-1)]*M
        #self.tree = self.makeRandTree(M, N)

    def makeZeroTree(self, M, N):
        if N == 1:
            return [0]*M
        else:
            return [self.makeZeroTree(M, N-1)] * M


    # first element is value at this location, the second is a list (the children)
    def makeRandTree(self, M, N):
        if N == 1:
            return [0.2*rand()]*M
        else:
            return [0.2*rand(), self.makeRandTree(M, N-1)] * M

    def makeCountedTree(self, M, N, count = 0):
        if N == 1:
            return list(range(count, count + M))
            count += M
        else:
            return [self.makeCountedTree(M, N-1, count)] * M

    def get_alpha(self, x):
        r = self
        for j in x:
            print(f"r: {r}")
            r = r.children[j]
            
        return r.alpha

    def loss(self, code, priors):
        return 0

    # takes a decision tree, and considers all possible decision tables (of which there are 2**2**M
    def gen_table(self):
        

In [16]:
# given an input state, a decision tree, and a measurement history, outputs the probability of that
# measurement history
def prob(tree, x, state):
    return 0

def loss(tree, states):
    return 0

In [64]:

M = 3# dimension of Hilbert space. This is a truncated Fock state, with the basis (|0>, |1>, ... |M-1>).
# Hilbert space has dimension 2**M
N = 3 # depth of circuit, including the root node
beta = 5 # this is a small parameter to ensure the coherent states are "weak"
gamma = 1e-3 # intensity of the injected states

# parameters of six coherent states to distinguish between.
code = {
    0: beta*exp(1j * atan(0.5)),
    1: beta*exp(-1j * atan(0.5))
}

# prior probabilities of the input states
priors = {}
for j in range(len(code)):
    priors[j] = 1/len(code)


projs = [qutip.states.ket2dm(qutip.states.basis(M, j)) for j in range(M)] # single-mode projectors
projs0 = [tensor(p, identity(M)) for p in projs]


tree2 = DecisionTree(M, N)
print("tree:")
print(f"alpha: {tree2.get_alpha([1, 0])}")

# given M and N, gives a list of length M**N, such that each element is a list of length M, containing
# ints between 0 and M-1
# gives every possible measurement set
def make_xs(M, N):
    if N == 1:
        return [[j] for j in range(M)]
    else:
        xsm = make_xs(M, N-1)
        res = []
        for xm in xsm:
            for j in range(M):
                xm.append(j)
                res.append(xm.copy())
                xm.pop()
        return res

print(make_xs(M, 1))
print(make_xs(M, 2))

# takes an input state, and a decision tree. yields the measurement history
def process(tree, N, M, code):
    tapped = coherent(M, code[0])
    x = []
    for j in range(N):
        alpha = tree.get_alpha(x)
        state = displace(tapped, alpha)
        [outcome, _] = measure(state, projs)
        x = x.append(outcome)

    return x

# takes a decision tree, and Q
# runs the circuit Q times with tree, and tells 
def table(tree, Q):
    table = {}
    meas = {}
    for _ in Q:
        code_in = code[randint(len(code))]
        outcome = process(tree, N)
        meas[outcome].append(code_in)

    for x in xs:
        posteriors = [meas[x].count(j)*priors(j) for j in len(code)]
        table[x] = posteriors.find(max(posteriors))

    loss = 0
    for x in xs:
        for i in range(len(code)):
            error = (table[x] != i)
            prob = meas[x]/Q
            prior = priors[i]
            loss += error*prob*prior



tree:
r: <__main__.DecisionTree object at 0x7b9a99a46ed0>
r: <__main__.DecisionTree object at 0x7b9a99a47bf0>
alpha: (0.47754886832068977+0.25219159009428416j)
[[0], [1], [2]]
[[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2]]


In [85]:
print(range(0, 3))

range(0, 3)


In [57]:
print(list(range(0, 3)))

[0, 1, 2]
